In [321]:

from lusidtools.jupyter_tools import toggle_code

"""Term Deposit Valuation

Demonstrates pricing of a Term Deposit Investment.

Attributes
----------
instruments
valuation
lifecycle events
market data store
results store
quotes
"""

toggle_code("Hide docstring")

## 1. Setup

In [322]:
# Import LUSID libraries
import lusid as lu
import lusid.models as lm
from lusidtools.lpt.lpt import to_date

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam.refreshing_token import RefreshingToken

# Import Libraries
from datetime import datetime, timedelta
import pytz
import pandas as pd
import json
import os

# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.11152.0


In [323]:
# Initiate the LUSID APIs required for the notebook
instruments_api = api_factory.build(lu.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
complex_market_data_api = api_factory.build(lu.api.ComplexMarketDataApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.AggregationApi)
structured_result_data_api = api_factory.build(lu.api.StructuredResultDataApi)

In [324]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"
portfolio_code = "TermDepositExamplePortfolio"

### 1.1 Create Portfolios

In [325]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'TermDepositExamplePortfolio' because it already exists in scope 'ibor'.


### 1.2 Create TD Instruments

In [326]:
flow_conventions = lm.FlowConventions(
    currency="GBP",
    payment_frequency="6M",
    roll_convention="F",
    day_count_convention="Actual360",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0,
)

start_date = datetime(2025, 1, 1, 00, tzinfo=pytz.utc)
maturity_date = datetime(2035, 1, 1, 00, tzinfo=pytz.utc)
rolling_date = maturity_date - timedelta(days=1)

def create_td_instrument(
    name,
    identifier,
    start_date,
    maturity_date,
    contract_size,
    flow_convention,
    rate,
    dom_ccy
):
    td_instrument = lm.TermDeposit(
        start_date=start_date,
        maturity_date=maturity_date,
        contract_size=contract_size,
        flow_convention=flow_convention,
        rate=rate,
        dom_ccy=dom_ccy,
        instrument_type="TermDeposit",
        local_vars_configuration=None)

    td_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(identifier)},
        definition=td_instrument
    )

    upsert_request = {identifier: td_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    print(upsert_response)
    td_luid = upsert_response.values[identifier]
    print(td_luid)

In [327]:
td_luid_a = create_td_instrument(
    "TD Example Instrument A",
    "TD_EXAMPLE_A",
    start_date,
    maturity_date,
    1000,
    flow_conventions,
    1.03,
    "GBP"
)
td_luid_a

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ1MG9QS42J:00000085',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Updated Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['TD_EXAMPLE_A'],
 'type': 'UpdatedInstruments'}]},
 'values': {'TD_EXAMPLE_A': {'asset_class': 'InterestRates',
                             'dom_ccy': 'GBP',
                             'href': 'https://liberty.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003DEN?scope=default',
                             'identif

In [328]:
td_luid_b = create_td_instrument(
    "TD Example Instrument B",
    "TD_EXAMPLE_B",
    start_date,
    maturity_date,
    1000,
    flow_conventions,
    1.05,
    "GBP"
)

td_luid_b

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ1MG9T1AIR:000001E1',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Updated Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['TD_EXAMPLE_B'],
 'type': 'UpdatedInstruments'}]},
 'values': {'TD_EXAMPLE_B': {'asset_class': 'InterestRates',
                             'dom_ccy': 'GBP',
                             'href': 'https://liberty.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003DEO?scope=default',
                             'identif

In [329]:
td_luid_c = create_td_instrument(
    "TD Example Instrument C",
    "TD_EXAMPLE_C",
    start_date,
    maturity_date,
    1000,
    flow_conventions,
    1.07,
    "GBP"
)

td_luid_c

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ1MFTDNHE7:00000009',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Unchanged Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['TD_EXAMPLE_C'],
 'type': 'UnchangedInstruments'}]},
 'values': {'TD_EXAMPLE_C': {'asset_class': 'InterestRates',
                             'dom_ccy': 'GBP',
                             'href': 'https://liberty.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003DEP?scope=default',
                             'ide

### 1.4 Load transactions into the repo

In [330]:
transactions_df = pd.read_csv("data/term_deposit_transactions.csv")
transactions_df.head()

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency
0,txn001,StockIn,TD_EXAMPLE_A,2020-01-01T00:00:00Z,2020-01-03T00:00:00Z,75,200,"15,000.00",USD
1,txn002,StockIn,TD_EXAMPLE_B,2020-01-01T00:00:00Z,2020-01-03T00:00:00Z,100,220,"22,000.00",USD
2,txn003,Sell,TD_EXAMPLE_A,2034-12-05T00:00:00Z,2034-12-08T00:00:00Z,75,200,"15,000.00",USD
3,txn004,StockIn,TD_EXAMPLE_A,2034-12-10T00:00:00Z,2034-12-13T00:00:00Z,75,150,"11,250.00",USD
4,txn005,Sell,TD_EXAMPLE_B,2034-12-15T00:00:00Z,2034-12-18T00:00:00Z,100,220,"22,000.00",USD


In [331]:
transaction_request = [
    lm.TransactionRequest(
        transaction_id=txn["txn_id"],
        type=txn["type"],
        instrument_identifiers={
            "Instrument/default/ClientInternal": txn["client_id"]
        },
        transaction_date=to_date(txn["trade_date"]).isoformat(),
        settlement_date=to_date(txn["settlement_date"]).isoformat(),
        units=txn["quantity"],
        transaction_price=lm.TransactionPrice(price=txn["price"], type="Price"),
        total_consideration=lm.CurrencyAndAmount(
            amount=txn["total_consideration"], currency=txn["currency"]
        ),
    )
    for index, txn in transactions_df.iterrows()
]


response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=transaction_request
)

print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2023-04-20 15:50:56.225796+00:00


## 2. Get Holdings

In [332]:
def get_holdings(date):
    resp = transaction_portfolios_api.get_holdings(scope=scope,
                                         code=portfolio_code,
                                         property_keys=["Instrument/default/Name"],
                                        effective_at=date)
    df = lusid_response_to_data_frame(resp)
    return df[["instrument_scope","instrument_uid","properties.Instrument/default/Name.value.label_value","units","cost.amount","holding_type_name"]]

In [333]:
# Before first roll
get_holdings(datetime(2034,12,1,00, tzinfo=pytz.utc))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,LUID_00003DEN,TD Example Instrument A,75.0000,"15,000.0000",Position
1,default,LUID_00003DEO,TD Example Instrument B,100.0000,"22,000.0000",Position


In [334]:
# After first roll
get_holdings(datetime(2034,12,10,00, tzinfo=pytz.utc))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,LUID_00003DEN,TD Example Instrument A,75.0000,"11,250.0000",Position
1,default,LUID_00003DEO,TD Example Instrument B,100.0000,"22,000.0000",Position
2,default,CCY_USD,USD,"15,000.0000","15,000.0000",Balance


In [335]:
# After second roll
get_holdings(datetime(2034,12,31,00, tzinfo=pytz.utc))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,LUID_00003DEN,TD Example Instrument A,75.0000,"11,250.0000",Position
1,default,CCY_USD,USD,"37,000.0000","37,000.0000",Balance
2,default,LUID_00003DEP,TD Example Instrument C,100.0000,"22,000.0000",Position


## 5. Running a Valuation
### 5.1 Setup recipe and valuations

In [336]:
# Define a method for creating the recipe
def create_recipe(recipe_code, scope, model):
    # Populate recipe parameters
    recipe = lm.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key="Equity.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="5D.0D"
                ),
                lm.MarketDataKeyRule(
                    key="FxForwards.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="1Y.0D"
                )
            ],
            options=lm.MarketOptions(
                attempt_to_infer_missing_fx=True,
                default_scope=scope
            )
        ),
        # Set the valuation model - curve with no discounting
        pricing=lm.PricingContext(
            model_rules=[
                lm.VendorModelRule(
                    supplier="Lusid",
                    model_name=model,
                    instrument_type="TermDeposit",
                    parameters="{}",
                )
            ],
            options=lm.PricingOptions(
                produce_separate_result_for_linear_otc_legs=False
            )
        ),
    )

    response = configuration_recipe_api.upsert_configuration_recipe(
                upsert_recipe_request=lm.UpsertRecipeRequest(
                    configuration_recipe=recipe
                )
            )

    return response

CTVoMRecipe = "TermDepositRecipeCTVoM"
SSRecipe = "TermDepositRecipeSS"

create_recipe(CTVoMRecipe, scope, "ConstantTimeValueOfMoney")
create_recipe(SSRecipe, scope, "SimpleStatic")

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ1MG9T1AM5:0000007F',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2023, 4, 20, 15, 50, 57, 890674, tzinfo=tzutc())}

In [350]:
# Run valuation
def perform_valuation(recipe, date):
    # Create valuation request
    valuation_request = lm.ValuationRequest(
        # Choose recipe to use
        recipe_id = lm.ResourceId(scope = scope, code = recipe),
        group_by = ["Instrument/default/LusidInstrumentId"],
        metrics = [
        {"key": "Instrument/default/Name", "op": "Value"},              # Reports the friendly name of the underlying instrument
        {"key": "Analytic/default/ValuationDate", "op": "Value"},       # Confirms the valuation date
        {"key": "Valuation/PV", "op": "Sum"}],                          # Calculates cost in GBP (the portfolio currency)
        # Identify portfolio to value
        portfolio_entity_ids = [lm.PortfolioEntityId(scope = scope, code = portfolio_code)],
        # Make date of valuation conditional on effective at date passed into function
        valuation_schedule = lm.ValuationSchedule(effective_at=date),
    )

    # Get portfolio valuation
    val_data = aggregation_api.get_valuation(valuation_request = valuation_request).data

    # Turn valuation response into pandas dataframe
    vals_df = pd.DataFrame(val_data)
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df


def get_instr_valuation(instrument, date):
    df = perform_valuation()

In [351]:
# Sell based on valuation
sell_td = lm.TransactionRequest(
    transaction_id="txn003",
    type="Sell",
    instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
    transaction_date=rolling_date.isoformat(),
    settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
    units=75,
    transaction_price=lm.TransactionPrice(price=0,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

### 5.2 Valuations using CTVoM

In [338]:
# Before first roll
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2034,12,1,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2034-12-01T00:00:00.0000000+00:00,"15,000.0000"
1,TD Example Instrument B,2034-12-01T00:00:00.0000000+00:00,"22,000.0000"


In [339]:
# After first roll
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2034,12,14,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2034-12-14T00:00:00.0000000+00:00,"11,250.0000"
1,TD Example Instrument B,2034-12-14T00:00:00.0000000+00:00,"22,000.0000"
2,USD,2034-12-14T00:00:00.0000000+00:00,"15,000.0000"


In [340]:
# After second roll
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2034,12,24,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2034-12-24T00:00:00.0000000+00:00,"11,250.0000"
1,USD,2034-12-24T00:00:00.0000000+00:00,"37,000.0000"
2,TD Example Instrument C,2034-12-24T00:00:00.0000000+00:00,"22,000.0000"


In [341]:
# After maturity
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2035,1,1,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2035-01-01T00:00:00.0000000+00:00,"11,250.0000"
1,USD,2035-01-01T00:00:00.0000000+00:00,"37,000.0000"
2,TD Example Instrument C,2035-01-01T00:00:00.0000000+00:00,"22,000.0000"


### 5.3 Valuations using SS

In [342]:
# Before first roll
perform_valuation(SSRecipe, portfolio_code,  datetime(2034,12,1,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2034-12-01T00:00:00.0000000+00:00,"15,000.0000"
1,TD Example Instrument B,2034-12-01T00:00:00.0000000+00:00,"22,000.0000"


In [343]:
# After first roll
perform_valuation(SSRecipe, portfolio_code,  datetime(2034,12,14,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2034-12-14T00:00:00.0000000+00:00,"11,250.0000"
1,TD Example Instrument B,2034-12-14T00:00:00.0000000+00:00,"22,000.0000"
2,USD,2034-12-14T00:00:00.0000000+00:00,"15,000.0000"


In [344]:
# After second roll
perform_valuation(SSRecipe, portfolio_code,  datetime(2034,12,24,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2034-12-24T00:00:00.0000000+00:00,"11,250.0000"
1,USD,2034-12-24T00:00:00.0000000+00:00,"37,000.0000"
2,TD Example Instrument C,2034-12-24T00:00:00.0000000+00:00,"22,000.0000"


In [345]:
# After maturity
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2035,1,1,00, tzinfo=pytz.utc))

,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Holding/Cost/Pfolio)
0,TD Example Instrument A,2035-01-01T00:00:00.0000000+00:00,"11,250.0000"
1,USD,2035-01-01T00:00:00.0000000+00:00,"37,000.0000"
2,TD Example Instrument C,2035-01-01T00:00:00.0000000+00:00,"22,000.0000"


? How do I feed valuation price into sale price
? Should valuation prices change with time
? Does the second purchase after the roll need to have a later maturity

In [346]:
# Book a sell transaction against the term deposit
preMaturityValuation = instA_valuation(rolling_date)

sell_td = lm.TransactionRequest(
    transaction_id="txn003",
    type="Sell",
    instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
    transaction_date=rolling_date.isoformat(),
    settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
    units=75,
    transaction_price=lm.TransactionPrice(price=0,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                    code=portfolio_code,
                                                    transaction_request=[sell_td])
response

In [347]:
# # Buy TD back with lower market value
#
# sell_td = lm.TransactionRequest(
#     transaction_id="txn004",
#     type="Sell",
#     instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
#     transaction_date=rolling_date.isoformat(),
#     settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
#     units=75,
#     transaction_price=lm.TransactionPrice(price=0,type="Price"),
#     total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
#     exchange_rate=1,
#     transaction_currency="USD"
# )
